In [ ]:
!pip uninstall -y numpy scipy autogluon autogluon.timeseries
!pip install numpy==1.23.5 scipy==1.9.3
!pip install autogluon.timeseries
!pip install torch

In [5]:
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import torch

# Load your preprocessed dataset
file_path = 'automl_dataset.csv'
df = pd.read_csv(file_path)

# Ensure timestamp column is datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Set all rows to the same item_id (1) to create a single time series
df['item_id'] = 1

# Convert to TimeSeriesDataFrame
df_tsd = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Split into training and testing data
train_size = int(len(df_tsd) * 0.8)
split_timestamp = df_tsd.index.get_level_values(1)[train_size]

train_data = df_tsd[df_tsd.index.get_level_values(1) < split_timestamp]
test_data = df_tsd[df_tsd.index.get_level_values(1) >= split_timestamp]

# Setup and train the TimeSeriesPredictor with reduced complexity
predictor = TimeSeriesPredictor(
    path='autogluon_checkpoints',
    prediction_length=1,
    
    eval_metric='RMSE',
    freq='D',
)

# Fit the model with reduced time limit and simpler preset
predictor.fit(
    train_data=train_data, 
    presets='high_quality',
    time_limit=9000, # 2,5 hours
    excluded_model_types=['RecursiveTabular', 'DirectTabular']
)

# Generate predictions on test data
predictions = predictor.predict(test_data)

# Save predictions
predictions.to_csv('autogluon_predictions.csv')

# Leaderboard of models
leaderboard = predictor.leaderboard()
print(leaderboard)

Beginning AutoGluon training... Time limit = 9000s
AutoGluon will save models to 'autogluon_checkpoints'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.0.0: Mon Aug 12 20:51:54 PDT 2024; root:xnu-11215.1.10~2/RELEASE_ARM64_T6000
CPU Count:          10
GPU Count:          0
Memory Avail:       3.25 GB / 16.00 GB (20.3%)
Disk Space Avail:   182.29 GB / 926.35 GB (19.7%)
Setting presets to: high_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': RMSE,
 'excluded_model_types': ['RecursiveTabular', 'DirectTabular'],
 'freq': 'D',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target

                       model  score_val  pred_time_val  fit_time_marginal  \
0           WeightedEnsemble  -5.154573       0.016153           0.273677   
1  TemporalFusionTransformer  -5.154573       0.016153         101.260629   
2              SeasonalNaive  -6.173333       1.815764           0.006769   
3                     DeepAR  -7.879876       0.007127          19.736750   
4      DynamicOptimizedTheta  -8.364108       9.926698           0.006751   
5                    AutoETS  -9.395426      10.546663           0.010367   
6                   PatchTST -11.012685       0.005308          19.474832   
7                  AutoARIMA -11.291668       5.712947           0.010273   
8                 CrostonSBA -13.859536       4.039534           0.006207   
9                       NPTS -23.730088       0.016415           0.005893   

   fit_order  
0         10  
1          7  
2          1  
3          8  
4          4  
5          5  
6          9  
7          6  
8          2  
9 